In [ ]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pytz openai

In [3]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="creds/.env")

True

In [4]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )


In [5]:
import time
import json
# from openai import OpenAI
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# client= OpenAI(
#     api_key = OPENAI_API_KEY
# )

agent_instructions = """
You are an intelligent assistant tasked with helping users manage their personal and professional schedules.
Your capabilities include creating, updating, and deleting Google Calendar events as per user directives.
Emphasize user privacy and data security in all your operations.
"""

agent_tools = [
  {"type": "code_interpreter"},
  {
    "type": "function",
    "function": {
      "name": "create_event",
      "description": "Create a new calendar event",
      "parameters": {
        "type": "object",
        "properties": {
          "event": {
            "type": "object",
            "description": "The event details including summary, location, description, start, and end times",
            "properties": {
              "summary": {"type": "string"},
              "location": {"type": "string"},
              "description": {"type": "string"},
              "start": {
                "type": "object",
                "properties": {
                  "dateTime": {"type": "string"},
                  "timeZone": {"type": "string"}
                },
                "required": ["dateTime", "timeZone"]
              },
              "end": {
                "type": "object",
                "properties": {
                  "dateTime": {"type": "string"},
                  "timeZone": {"type": "string"}
                },
                "required": ["dateTime", "timeZone"]
              }
            },
            "required": ["summary", "start", "end"]
          }
        },
        "required": ["event"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_event",
      "description": "Retrieve details of a specific event by its ID",
      "parameters": {
        "type": "object",
        "properties": {
          "event_id": {"type": "string", "description": "The ID of the event to retrieve"}
        },
        "required": ["event_id"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "update_event",
      "description": "Update an existing calendar event",
      "parameters": {
        "type": "object",
        "properties": {
          "event_id": {"type": "string", "description": "The ID of the event to update"},
          "updated_event": {
            "type": "object",
            "description": "The updated event details",
            "properties": {
              "summary": {"type": "string"},
              "location": {"type": "string"},
              "description": {"type": "string"},
              "start": {
                "type": "object",
                "properties": {
                  "dateTime": {"type": "string"},
                  "timeZone": {"type": "string"}
                },
                "required": ["dateTime", "timeZone"]
              },
              "end": {
                "type": "object",
                "properties": {
                  "dateTime": {"type": "string"},
                  "timeZone": {"type": "string"}
                },
                "required": ["dateTime", "timeZone"]
              }
            },
            "required": ["start", "end"]
          }
        },
        "required": ["event_id", "updated_event"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "delete_event",
      "description": "Delete a specific event by its ID",
      "parameters": {
        "type": "object",
        "properties": {
          "event_id": {"type": "string", "description": "The ID of the event to delete"}
        },
        "required": ["event_id"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_conflicts",
      "description": "Check for any events occurring during a given time period",
      "parameters": {
        "type": "object",
        "properties": {
          "start_time": {"type": "string", "description": "The start time of the period (RFC3339 format)"},
          "end_time": {"type": "string", "description": "The end time of the period (RFC3339 format)"}
        },
        "required": ["start_time", "end_time"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_events",
      "description": "List all events occurring within a given time period",
      "parameters": {
        "type": "object",
        "properties": {
          "start_time": {"type": "string", "description": "The start time of the period (RFC3339 format)"},
          "end_time": {"type": "string", "description": "The end time of the period (RFC3339 format)"}
        },
        "required": ["start_time", "end_time"]
      }
    }
  },
  {
    "type": "function",
    "function": {
        "name": "get_current_datetime_and_timezone",
        "description": "Get the current date and time with timezone information",
        "parameters": {}
    }
  }
]

# assistant = client.beta.assistants.create(
#     name="Data Visualization",
#     instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
#     f"You have access to a sandboxed environment for writing and testing code."
#     f"When you are asked to create a visualization you should follow these steps:"
#     f"1. Write the code."
#     f"2. Anytime you write new code display a preview of the code to show your work."
#     f"3. Run the code to confirm that it runs."
#     f"4. If the code is successful display the visualization."
#     f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
#     tools=[{"type": "code_interpreter"}],
# )

assistant = client.beta.assistants.create(
  name="Scheduler Agent",
  instructions=agent_instructions,
  model="cursor-gpt-4", #You must replace this value with the deployment name for your model.
  # model="pjf-dpo-turbo-35", #You must replace this value with the deployment name for your model.
  tools=agent_tools,
)

In [6]:
def execute_function(calendar_service, name, args_dict):

    method = getattr(calendar_service, name, None)

    if not method:
        raise ValueError(f"Method '{name}' not found in CalendarService class.")

    return method(**args_dict)

def check_run_status(client, thread_id, run_id, calendar_service):
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

        if run.status in ['completed', 'failed', 'cancelled', 'expired']:
            return run.status
        elif run.status == 'requires_action':

            tool_outputs = []
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_id = tool_call.id
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)

                try:
                    output = execute_function(calendar_service, function_name, arguments)
                    tool_outputs.append({
                        "tool_call_id": tool_id,
                        "output": json.dumps(output)
                    })
                except Exception as e:
                    print(f"Error executing tool {function_name}: {e}")

                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id,
                    run_id=run_id,
                    tool_outputs=tool_outputs
                )

                time.sleep(0.1)
            else:
                time.sleep(0.1)

In [7]:
def start_chat(client, assistant_id, calendar_service):

    thread = client.beta.threads.create()

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Assistant deactivated.")
            break

        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )

        run = client.beta.threads.runs.create(
          thread_id=thread.id,
          assistant_id=assistant_id,
        )

        check_run_status(client, thread.id, run.id, calendar_service)

        thread_messages = client.beta.threads.messages.list(thread.id, order="desc", limit=1)
        if thread_messages.data:
            latest_message = thread_messages.data[0]
            if latest_message.role == "assistant":
                print(f"Assistant: {latest_message.content[0].text.value}")

## Google Calendar API

In [8]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = service_account.Credentials.from_service_account_file(
        "/Users/ogb/projects/cal/creds/gcal-ai.json", scopes=SCOPES)
  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="ongjjbrandon@gmail.com",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Getting the upcoming 10 events
2024-02-22T18:00:00+08:00 Therapy 6PM
2024-02-22T19:15:00+08:00 715PM Bwee
2024-02-23 Prof Misra 2PM SMU Admin Building
2024-02-24 Rush
2024-02-25 Bday dinner
2024-02-27T18:00:00+08:00 6 Chiro
2024-02-28 Bheng
2024-02-28T14:00:00+08:00 <Physical> Meeting on NLP/NLU/LLM (Brandon Ong)
2024-02-29 Dune
2024-02-29 Pay Citi Bills By


In [9]:
import pytz
pytz.country_names['sg']

'Singapore'

In [10]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime
import pytz


SERVICE_ACCOUNT_FILE = '/Users/ogb/projects/cal/creds/gcal-ai.json'

class CalendarService:
    def __init__(self, service_account_file, calendar_id='primary',timezone='Singapore'):
        self.service_account_file = service_account_file
        self.calendar_id = calendar_id
        self.calendar_service = self.get_calendar_service()
        self.timezone = timezone

    def get_calendar_service(self):
        credentials = service_account.Credentials.from_service_account_file(
            self.service_account_file, scopes=['https://www.googleapis.com/auth/calendar'])
        return build('calendar', 'v3', credentials=credentials)

    def create_event(self, event):
        event_result = self.calendar_service.events().insert(calendarId=self.calendar_id, body=event).execute()
        return event_result

    def get_event(self, event_id):
        event = self.calendar_service.events().get(calendarId=self.calendar_id, eventId=event_id).execute()
        return event

    def update_event(self, event_id, updated_event):
        event = self.calendar_service.events().update(calendarId=self.calendar_id, eventId=event_id, body=updated_event).execute()
        return event

    def delete_event(self, event_id):
        return self.calendar_service.events().delete(calendarId=self.calendar_id, eventId=event_id).execute()

    def get_conflicts(self, start_time, end_time):
        events_result = self.calendar_service.events().list(
            calendarId=self.calendar_id,
            timeMin=start_time,
            timeMax=end_time,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        return events

    def get_events(self, start_time, end_time):
        events_result = self.calendar_service.events().list(
            calendarId=self.calendar_id,
            timeMin=start_time,
            timeMax=end_time,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        return events

    def add_calendar_owner(self, user_email):
      calendar_id = 'primary'
      rule = {
          'scope': {
              'type': 'user',
              'value': user_email,
          },
          'role': 'owner'
      }
      created_rule = self.calendar_service.acl().insert(calendarId=calendar_id, body=rule).execute()
      return created_rule

    def get_current_datetime_and_timezone(self):

        timezone_str = self.timezone
        timezone = pytz.timezone(timezone_str)

        now = datetime.now(timezone)

        datetime_str = now.strftime('%Y-%m-%d %H:%M:%S %Z%z')

        return {
            "datetime": datetime_str,
            "timezone": timezone_str
        }

calendar_service = CalendarService(SERVICE_ACCOUNT_FILE, calendar_id='ongjjbrandon@gmail.com')


def execute_function(calendar_service, name, args_dict):

    method = getattr(calendar_service, name, None)

    if not method:
        raise ValueError(f"Method '{name}' not found in CalendarService class.")

    return method(**args_dict)


# # Define the event to add
# event = {
#   'summary': 'Google I/O 2025',
#   'location': '800 Howard St., San Francisco, CA 94103',
#   'description': 'A chance to hear more about Google\'s developer products.',
#   'start': {
#     'dateTime': '2025-05-28T09:00:00-07:00',
#     'timeZone': 'America/Los_Angeles',
#   },
#   'end': {
#     'dateTime': '2025-05-28T17:00:00-07:00',
#     'timeZone': 'America/Los_Angeles',
#   },
#   'reminders': {
#     'useDefault': False,
#     'overrides': [
#       {'method': 'email', 'minutes': 24 * 60},
#       {'method': 'popup', 'minutes': 10},
#     ],
#   },
# }


In [11]:
start_chat(client, assistant.id, calendar_service)

Assistant: Here are your events for today:

1. Event: Therapy
   - Time: 6:00 PM - 6:30 PM
   - [Event Link](https://www.google.com/calendar/event?eid=YXZpYWozN3Y0cjZ1bmFoZGRobzdydm4wdThfMjAyNDAyMjIgb25nampicmFuZG9uQG0)

2. Event: 715PM Bwee
   - Time: 7:15 PM - 7:45 PM
   - [Event Link](https://www.google.com/calendar/event?eid=NGMxMWUyN2Y5OWUxNDQ5NWExZDdmZjFiMTUwMTFlZjkgb25nampicmFuZG9uQG0)
Assistant: Here are your events for next week:

1. Event: 6 Chiro
   - Date: Feb 27 (Wed)
   - Time: 6:00 PM - 7:00 PM
   - [Event Link](https://www.google.com/calendar/event?eid=bmtyaDAwZWJuYWFmcDNscDhtcW0wcWxoc2hfMjAyNDAyMjhUMTAwMDAwWiBvbmdqamJyYW5kb25AbQ)
 
2. Event: Bheng
   - Date: Feb 28 (Wed)
   - Time: All day event
   - [Event Link](https://www.google.com/calendar/event?eid=ZXNqNmxnbzNiYjE1ZDNtbm5mZGtrM29nNnMgb25nampicmFuZG9uQG0)

3. Event: Meeting on NLP/NLU/LLM (Brandon Ong)
   - Date: Feb 28 (Wed)
   - Time: 2:00 PM - 3:00 PM
   - Location: Shannon meeting room, 12th level, NORTH CONNE

KeyboardInterrupt: 